# SQA data joiner

In [1]:
# Import librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# Functions

In [2]:
# Define functions to read sheets

def read_sheet_new(year, level, header):

    # read in xlsx
    df = pd.read_excel('./data_stats_2022/attainment-statistics-december-' + year +'.xlsx', 
                        sheet_name = level)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2022/attainment-statistics-december-' + year +'.xlsx', 
                        sheet_name = level, skiprows = header_index + 1)
    
    return df

def read_single_sheet(year, level, header, sheet_to_find):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_2000/ASR' + year + '_' + level + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)
    
    # feed this index into pandas
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = index)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = index, skiprows = header_index + 1)
    
    return df

def read_sheet(year, level, header, sheets_to_find):
        
    # create list of sheetnames
    tabs = pd.ExcelFile('./data_stats_2000/ASR' + year + '_' + level + '.xls').sheet_names
    # Use key to find correct substring to search sheetnames for
    sheets = sheets_to_find
    # Find correct sheetname, case insensitive search
    name = [s for s in tabs if any(tabs in s.lower() for tabs in sheets)][0]

    # read in xls
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = name)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = name, skiprows = header_index + 1)
    
    return df

def read_old_single_sheet(year, header, sheet_to_find, skip):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_1986/Stats' + year + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)

    # feed this index into pandas
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index, skiprows = header_index + skip)
    
    return df

def read_old_higher_single_sheet(year, header, sheet_to_find):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_1986/Stats' + year + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)

    # feed this index into pandas
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index)
    
    # Delete first column
    del df[df.columns[0]]
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index, skiprows = header_index + 3)
    

    # Delete first column
    del df[df.columns[0]]
    
    return df

In [3]:
# Define functions to tidy data

def tidy_df(df, footer):
    
    # Replace weird values with NaN
    df = df.replace('***', np.nan)
    df = df.replace('-', np.nan)
    df = df.replace(' -', np.nan)
    
    # Change columns to lower case
    df.columns = df.columns.astype(str).str.lower()
    
    # Rename first column to subject
    df.rename(columns = {df.columns[0]:'subject'}, inplace = True)

    # Drop rows with NaN in subject column
    df = df.dropna(subset = ['subject'])
    
    # Reset index
    df.reset_index(drop = True, inplace = True)

    # Search for footer in column and get index  
    footer_index = df[df.iloc[:, 0].str.match(footer, na = False, case = False)].index[0]
    # Drop all rows under footer column
    df = df.drop(df.index[footer_index:])
    
    if 'comp' in df.columns.values:
        df.rename(columns = {'comp':'d'}, inplace = True)
    
    # Remove whitespace from the end of all subjects
    df['subject'] = df['subject'].str.strip()
    
    # Update to recognise datatypes
    df = df.infer_objects()
    
    return df

In [4]:
# Define functions to count grades

def grade_count_new(df):
    
    # Change columns from percentages to counts
    df['a'] = df['grade a count 2022']
    df['b'] = df['grade a-b count 2022'] - df['grade a count 2022']
    df['c'] = df['grade a-c count 2022'] - df['grade a-b count 2022']
    df['d'] = df['grade a-d count 2022'] - df['grade a-c count 2022']
    
    return df

def grade_count(df):
    
    # Change columns from percentages to counts
    df['a'] = df['a'] * df['entries'] // 100
    df['b'] = df['b'] * df['entries'] // 100
    df['c'] = df['c'] * df['entries'] // 100
    df['d'] = df['d'] * df['entries'] // 100
    
    return df

def pass_fail(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['na'] = df['entries'] - df['a'] - df['b'] - df['c'] - df['d']
    
    # Add column with pass count
    df['pass'] = df['entries'] - df['na']
    
    return df

def pass_fail_no_d(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['na'] = df['entries'] - df['a'] - df['b'] - df['c']
    
    # Add column with pass count
    df['pass'] = df['entries'] - df['na']
    
    return df

def pass_fail_ordinary(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['na'] = df['entries'] - df['1'] - df['2'] - df['3'] - df['4'] - df['5']
    
    # Add column with pass count
    df['pass'] = df['entries'] - df['na']
    
    return df

In [5]:
# Define functions to select columns

def select_old_columns(df):
    
    # Select total entries and grades columns
    ls = ['subject', 'entries.2', '1.2', '2.2', '3.2', '4.2', '5.2', '6.2', '7.2']
    df = df[ls]
    
    # Rename total entries and grades columns
    di = {'entries.2': 'entries', '1.2': '1', '2.2': '2', '3.2': '3', '4.2': '4', '5.2': '5',
                                    '6.2': '6', '7.2': '7'}
    df = df.rename(columns = di)
    
    return df

def select_old_higher_columns(df):
    
    # Select total entries and grades columns
    ls = ['subject', 'entries.2', 'a.2', 'b.2', 'c.2', 'd.2']
    df = df[ls]
    
    # Rename total entries and grades columns
    di = {'entries.2': 'entries', 'a.2': 'a', 'b.2': 'b', 'c.2': 'c', 'd.2': 'd'}
    df = df.rename(columns = di)
    
    # Convert strs in dodgy rows in entries column to NaN
    df['entries'] = pd.to_numeric(df['entries'], errors = 'coerce')
    
    return df

def select_old_ordinary_columns(df):
    
    # Select total entries and grades columns
    ls = ['subject', 'entries.2', '1.2', '2.2', '3.2', '4.2', '5.2']
    df = df[ls]
    
    # Rename total entries and grades columns
    di = {'entries.2': 'entries', '1.2': '1', '2.2': '2', '3.2': '3', '4.2': '4', '5.2': '5'}
    df = df.rename(columns = di)
    
    return df

In [6]:
# Define functions to tidy up data at the end

# Function to combine different versions of the same subject (level)
def combine_subs_level(df, mainlevel, subs):
    
    # Create empty dataframes
    dfs = pd.DataFrame()
    dfu = pd.DataFrame()
    
    # Create empty list for dfs
    frames = []
    
    # Loop through list items
    for l in subs:
        
        # Find rows with level equal to other version of subject levels
        dfw = df[(df['level'] == l)]
        
        # Append to df list
        frames.append(dfw)
        
    # Concat list of dfs together    
    dfs = pd.concat(frames)
    
    # Change level value to main value
    dfs['level'] = mainlevel
        
    # Merge list of other versions to main df and fill NaNs with zeros
    dfu = pd.merge(df, dfs, how = 'outer', on = ['subject', 'grade', 'year', 'level'])
    dfu = dfu.fillna(0)
    
    # Add together two counts for each row
    dfu['count'] = dfu['count_x'] + dfu['count_y']
    
    # Drop working columns
    dfu = dfu.drop(['count_x', 'count_y'], axis=1)
    
    # Drop rows with other versions
    values = subs
    dfu = dfu.loc[~dfu['level'].isin(values)]
    
    return dfu

# Function to combine different versions of the same subject (subject name)
def combine_subs_name(df, mainname, subs):
    
    # Create empty dataframes
    dfs = pd.DataFrame()
    dfu = pd.DataFrame()
    
    # Create empty list for dfs
    frames = []
    
    # Loop through list items
    for n in subs:
        
        # Find rows with level equal to other version of subject levels
        dfw = df[(df['subject'] == n)]
        
        # Append to df list
        frames.append(dfw)
        
    # Concat list of dfs together    
    dfs = pd.concat(frames)
    
    # Change level value to main value
    dfs['subject'] = mainname
        
    # Add together duplicate rows
    dfs = dfs.groupby(['subject', 'grade', 'year', 'level'], as_index = False)['count'].sum()
    
    # Merge list of other versions to main df and fill NaNs with zeros
    dfu = pd.merge(df, dfs, how = 'outer', on = ['subject', 'grade', 'year', 'level'])
    dfu = dfu.fillna(0)
    
    # Add together two counts for each row
    dfu['count'] = dfu['count_x'] + dfu['count_y']
    
    # Drop working columns
    dfu = dfu.drop(['count_x', 'count_y'], axis=1)
    
    # Drop rows with other versions
    values = subs
    dfu = dfu.loc[~dfu['subject'].isin(values)]
    
    return dfu

# Read in and process excel files

In [11]:
# Create empty list for dfs
frames = []

In [12]:
# Define level and years
levels = ['National_4']
years = ['2022']

# Define header and footer to search for
header = 'subject'
footer = 'total'

# Create empty dataframe
dfl = pd.DataFrame()

# Loop through levels
for level in levels:
    
    # Loop through years
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df, footer)
        
        # Create a df for entries
        df1 = df[['subject','awarded count 2022']].copy()
        # Rename headers
        df1.columns = ['subject', 'count']
        # Add in year and level columns
        df1['year'] = int(year)
        df1['level'] = level
        df1['grade'] = 'pass'
        # Append to df list
        frames.append(df1)
        
        # Create a df for passes
        df2 = df[['subject','entries 2022']].copy()
        # Rename headers
        df2.columns = ['subject', 'count']
        # Add in year and level columns
        df2['year'] = int(year)
        df2['level'] = level
        df2['grade'] = 'entries'
        # Append to df list
        frames.append(df2)
        
        # Create a df for fails
        df3 = df[['subject','entries 2022', 'awarded count 2022']].copy()
        # Calculate number of no awards
        df3['count'] = df3['entries 2022'] - df3['awarded count 2022']
        # Drop headers used for calculation
        df3 = df3.drop(['entries 2022', 'awarded count 2022'], axis=1)
        # Add in year and level columns
        df3['year'] = int(year)
        df3['level'] = level
        df3['grade'] = 'na'
        # Append to df list
        frames.append(df3)

In [15]:
# Define level and years
levels = ['National_5', 'Higher', 'Advanced_Higher']
years = ['2022']

# Define header and footer to search for
header = 'subject'
footer = 'total'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df, footer)
        
        # Select important columns
        df = df.filter(regex='subject|2022')
        
        # Copy entries over to new column
        df['entries'] = df['entries 2022']
        
        # Calculate grade counts from percentages
        df = grade_count_new(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [16]:
# Define level and years
levels = ['National_5', 'Higher', 'Advanced_Higher']

years = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']

# Define sheetnames to grab and header, columns and footer to search for
sheets = ['table 4','4b']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()


for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate grade counts from percentages
        df = grade_count(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [17]:
level = 'New_Higher'
year = '2015'

sheet = 'table 4'
header = 'title'
footer = 'total'

# Create empty dataframe
dfl = pd.DataFrame()

# Read and tidy data
df = read_single_sheet(year, level, header, sheet)
df = tidy_df(df, footer)

# Calculate grade counts from percentages
df = grade_count(df)

# Calculate pass and fails
df = pass_fail(df)

# Change into long format
dfl = pd.melt(df, id_vars =['subject'], 
        value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                var_name ='grade', value_name ='count')

# Add in year and level columns
dfl['year'] = int(year)
dfl['level'] = level

# Append to df list
frames.append(dfl)

In [18]:
# Define level and years
levels = ['Intermediate2', 'Intermediate1']

years = ['2015', '2014']

# Define sheetnames to grab and header, columns and footer to search for
sheets = ['table 4','4b']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate grade counts from percentages
        df = grade_count(df)
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [19]:
# Define level and years
levels = ['Intermediate2', 'Intermediate1', 'Higher', 'Advanced_Higher']
years = ['2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        ### Look at graded courses
        if 'passes in ungraded courses' in df.columns.values:
            df1 = df[pd.isnull(df['passes in ungraded courses'])].copy()
        else:
            df1 = df
        
        # Calculate pass and fails
        df1 = pass_fail(df1)

        # Change into long format
        dfl = pd.melt(df1, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level

        
        # Append to df list
        frames.append(dfl)
        
        
        ### Look at ungraded courses
        if 'passes in ungraded courses' in df.columns.values:
            
            # Drop rows with grades
            df2 = df[pd.notnull(df['passes in ungraded courses'])].copy()
            
            # Sort column name
            df2.rename(columns = {'passes in ungraded courses': 'pass'}, inplace = True)
        
            # Add column with na count
            df2['na'] = df['entries'] - df2['pass']

            # Change into long format
            dfl = pd.melt(df2, id_vars =['subject'], 
                    value_vars = ['entries', 'pass', 'na'],
                            var_name ='grade', value_name ='count')

            # Add in year and level columns
            dfl['year'] = int(year)
            dfl['level'] = level
            
            # Append to df list
            frames.append(dfl)

In [20]:
# Define level and years
levels = ['Advanced_Higher']
years = ['2003', '2002', '2001']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [21]:
# Define level and years
levels = ['Intermediate1']
years = ['2003', '2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate pass and fails
        df = pass_fail_no_d(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [22]:
# Define level and years
levels = ['Intermediate2']
years = ['2003', '2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [23]:
# Define level and years
levels = ['Higher']
years = ['2003']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level

        # Append to df list
        frames.append(dfl)

In [24]:
# Define level and years
levels = ['Higher']
years = ['2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['3a']
header = 'title|subject'
footer = 'totals|total|subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level

        # Append to df list
        frames.append(dfl)

In [25]:
real_level = 'Old_Higher'
level = 'Standard_Grade' # Stats hidden away in SG document
years = ['2001', '2000']

sheet = 'hg3a'
header = 'subject'
footer = 'total|subtotals'

for year in years:
    
    # Read and tidy data
    df = read_single_sheet(year, level, header, sheet)
    df = tidy_df(df, footer)

    # Calculate pass and fails
    df = pass_fail(df)

    # Change into long format
    dfl = pd.melt(df, id_vars =['subject'], 
            value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                    var_name ='grade', value_name ='count')

    # Add in year and level columns
    dfl['year'] = int(year)
    dfl['level'] = real_level
    
    # Append to df list
    frames.append(dfl)

In [26]:
# Define level and years
level = 'National_4'
years = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']

# Define sheetnames to grab and header to search for
sheet1 = 'table 1'
sheet2 = 'table 2'
header = 'TITLE'
footer = 'Total'

# Create empty dataframe
dfl = pd.DataFrame()

for year in years:
        
    ### read in first sheet

    # Read and tidy data
    df1 = read_single_sheet(year, level, header, sheet1)
    df1 = tidy_df(df1, footer)

    # Select correct columns
    df1 = df1[['subject', year]]

    # Sort headers
    df1.columns = ['subject', 'count']

    # Add in year and level columns
    df1['year'] = int(year)
    df1['level'] = level
    df1['grade'] = 'entries'

    # Append to df list
    frames.append(df1)

    ### read in second sheet

    # Read and tidy data
    df2 = read_single_sheet(year, level, header, sheet2)
    df2 = tidy_df(df2, footer)

    # Select correct columns
    df2 = df2[['subject', year]]

    # Sort headers
    df2.columns = ['subject', 'count']

    # Add in year and level columns
    df2['year'] = int(year)
    df2['level'] = level
    df2['grade'] = 'pass'

    # Append to df list
    frames.append(df2)

    ### Calculate NAs

    # Copy first orginal df
    df3 = df1.copy()

    # Change grade to b - represents NAs
    df3['grade'] = 'na'

    # Change count to difference between entries and passes
    df3['count'] = df1['count'] - df2['count']

    # Append to df list
    frames.append(df3)

In [27]:
# Define level and years
levels = ['Standard_Grade']
years = ['2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', 
                 '2005', '2004', '2003']

# Define sheetnames to grab and header to search for
sheets = ['sg3']
header = 'title|subject'
footer = 'subtotal|totals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df, footer)
        
        # Replace nans with zeros
        df = df.fillna(0)
        
        # Replace grade 7 with no award
        df['na'] = df['7']
        
        # Add column with pass count
        df['pass'] = df['entries'] - df['na']
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['subject'], 
                value_vars = ['entries', '1', '2', '3', '4', '5', '6', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)

In [28]:
# Define level and years
levels = ['Standard_Grade']
years = ['2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheet1 = 'sg4a'
sheet2 = 'sg4b'
header = 'subject'
footer = 'subtotals'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        ### read in first sheet (males)
        
        # Read and tidy data
        df1 = read_single_sheet(year, level, header, sheet1)
        df1 = tidy_df(df1, footer)
        
        # Replace nans with zeros
        df = df.fillna(0)
        
        # Replace grade 7 with no award
        df1['na'] = df1['7']
        
        # Add column with pass count
        df1['pass'] = df1['entries'] - df1['na']
        
        
        ### read in second sheet (females)
        
        # Read and tidy data
        df2 = read_single_sheet(year, level, header, sheet2)
        df2 = tidy_df(df2, footer)
        
        # Replace grade 7 with no award
        df2['na'] = df2['7']
        
        # Add column with pass count
        df2['pass'] = df2['entries'] - df1['na']
        
        
        ### Add together values
        df3 = df1
        
        df3['entries'] =  df1['entries'] + df2['entries']
        df3['pass'] =  df1['pass'] + df2['pass']
        df3['na'] =  df1['na'] + df2['na']
        
        df3['1'] =  df1['1'] + df2['1']
        df3['2'] =  df1['2'] + df2['2']
        df3['3'] =  df1['3'] + df2['3']
        df3['4'] =  df1['4'] + df2['4']
        df3['5'] =  df1['5'] + df2['5']
        df3['6'] =  df1['6'] + df2['6']
        
        # Change into long format
        dfl = pd.melt(df3, id_vars =['subject'], 
                value_vars = ['entries', '1', '2', '3', '4', '5', '6', 'pass', 'na'],
                        var_name ='grade', value_name ='count')
        
        
        # Add in year and level columns
        dfl['year'] = int(year)
        dfl['level'] = level
        
        # Append to df list
        frames.append(dfl)
        
#         # Rough conversion to N5 grades idea - 1,2 > a, 3,4 > b, 5 > c, 6 > d, 7 > na
#         dfs['a'] = dfs['1'] + dfs['2']
#         dfs['b'] = dfs['3'] + dfs['4']
#         dfs['c'] = dfs['5']
#         dfs['d'] = dfs['6']
#         dfs['na'] = dfs['7']

In [29]:
years = ['1999', '1998', '1997', '1996', '1995', '1994', '1993',
                    '1992', '1991', '1990', '1989', '1988', '1987', '1986']

level = 'Standard_Grade'

# Define sheetnames to grab and header to search for
sheet = 'std grade'
header = 'subject'
footer = 'totals'

for year in years:
    
    # Read and tidy data
    df = read_old_single_sheet(year, header, sheet, 2)
    df = tidy_df(df, footer)
    df = select_old_columns(df)
    
    # Replace nans with zeros
    df = df.fillna(0)

    # Replace grade 7 with no award
    df['na'] = df['7']

    # Add column with pass count
    df['pass'] = df['entries'] - df['na']

    # Change into long format
    dfl = pd.melt(df, id_vars =['subject'], 
            value_vars = ['entries', '1', '2', '3', '4', '5', '6', 'pass', 'na'],
                    var_name ='grade', value_name ='count')


    # Add in year and level columns
    dfl['year'] = int(year)
    dfl['level'] = level
    
    # Append to df list
    frames.append(dfl)

In [30]:
years = ['1999', '1998', '1997', '1996', '1995', '1994', '1993',
                    '1992', '1991', '1990', '1989', '1988', '1987', '1986']

level = 'Higher'

# Define sheetnames to grab and header to search for
sheet = 'higher'
header = 'subject'
footer = 'totals'

for year in years:
    
    # Read and tidy data
    df = read_old_higher_single_sheet(year, header, sheet)
    df = tidy_df(df, footer)
    df = select_old_higher_columns(df)

    # Calculate pass and fails
    df = pass_fail(df)
    
    # Change into long format
    dfl = pd.melt(df, id_vars =['subject'], 
            value_vars = ['entries', 'a', 'b', 'c', 'd', 'pass', 'na'],
                    var_name ='grade', value_name ='count')


    # Add in year and level columns
    dfl['year'] = int(year)
    dfl['level'] = level
    
    # Append to df list
    frames.append(dfl)

In [31]:
years = ['1988', '1987', '1986']

level = 'Ordinary_Grade'

# Define sheetnames to grab and header to search for
sheet = 'ordinary grade'
header = 'subject'
footer = 'totals'

for year in years:
    
    # Read and tidy data
    df = read_old_single_sheet(year, header, sheet, 3)
    df = tidy_df(df, footer)
    df = select_old_ordinary_columns(df)
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    df = pass_fail_ordinary(df)

    # Change into long format
    dfl = pd.melt(df, id_vars =['subject'], 
            value_vars = ['entries', '1', '2', '3', '4', '5', 'pass', 'na'],
                    var_name ='grade', value_name ='count')


    # Add in year and level columns
    dfl['year'] = int(year)
    dfl['level'] = level
    
    # Append to df list
    frames.append(dfl)

# Tidy up

In [32]:
# Concat list of dfs together
dft = pd.concat(frames)

# Remove whitespace from the end of values in subject column
dft['subject'] = dft['subject'].str.strip()

# Replace dodgy substrings in subject column in DataFrame
dft['subject'] = dft['subject'].str.replace(' - ', ': ')
dft['subject'] = dft['subject'].str.replace(' : ', ': ')
dft['subject'] = dft['subject'].str.replace('  ', ' ')
dft['subject'] = dft['subject'].str.replace(' & ', ' and ')

dft.head()

,subject,count,year,level,grade
0,Administration and IT,1900.0,2022,National_4,pass
1,Applications of Mathematics,6130.0,2022,National_4,pass
2,Art and Design,2895.0,2022,National_4,pass
3,Biology,7845.0,2022,National_4,pass
4,Business,3095.0,2022,National_4,pass


In [33]:
# TO DO
#n/a > np.nan??

# def tidy_up_subjects(df):

#     # Update names
#     di = {,
#             'Health and Food Technology': 'Home Economics: Health and Food Technology',
#             'Product Design': 'Design and Manufacture',
#             'Food Production Supervision': 'Hospitality: Food Production Supervision',
#             'Creative Cake Production': 'Hospitality: Practical Cake Craft',
#             'Craft and Design': 'Craft & Design',
#             'Woodworking Skills': 'Practical Woodworking',
#             'Applied Practical Electronics': 'Practical Electronics',
#             'Fabrication and Welding Engineering': 'Fabrication and Welding',
#             'Hairdressing: Composite Skills in Hairdressing': 'Hairdressing',
#              }
    
#     df = df.replace({"subject": di})
    
#     # Remove writing parts of language courses
#     values = ['German (Writing)*', 'French  (Writing)*', 'Spanish (Writing)*']
#     df = df.loc[~df['subject'].isin(values)]
    
#     return df

# # Tidy up subjects
# dft = tidy_up_subjects(dft)

# Tidy up some more

In [34]:
# Define updated df to work on for graphs
dfu = dft.copy()

In [35]:
subls = sorted(dfu['subject'].unique())

tag = ' (Revised)'

# Look for items with tag in the namme
matches = [match for match in subls if tag in match]

# Remove this from the name
matches_clean = [s.replace(tag, '') for s in matches]

# Create dictionary of the two lists above
di = dict(zip(matches, matches_clean))

# Loop through dictionary and call function to combine different versions of subject

for k in di:
    
    subs = [k]
    
    dfu = combine_subs_name(dfu, di[k], subs)

In [36]:
subls = sorted(dfu['subject'].unique())

tag = ' (Unrevised)'

# Look for items with tag in the namme
matches = [match for match in subls if tag in match]

# Remove this from the name
matches_clean = [s.replace(tag, '') for s in matches]

# Create dictionary of the two lists above
di = dict(zip(matches, matches_clean))

# Loop through dictionary and call function to combine different versions of subject

for k in di:
    
    subs = [k]
    
    dfu = combine_subs_name(dfu, di[k], subs)

In [37]:
subls = sorted(dfu['subject'].unique())

tag = ' (New)'

# Look for items with tag in the namme
matches = [match for match in subls if tag in match]

# Remove this from the name
matches_clean = [s.replace(tag, '') for s in matches]

# Create dictionary of the two lists above
di = dict(zip(matches, matches_clean))

# Loop through dictionary and call function to combine different versions of subject

for k in di:
    
    subs = [k]
    
    dfu = combine_subs_name(dfu, di[k], subs)

In [38]:
subls = sorted(dfu['subject'].unique())

tag = '*'

# Look for items with tag in the namme
matches = [match for match in subls if tag in match]

# Remove this from the name
matches_clean = [s.replace(tag, '') for s in matches]

# Create dictionary of the two lists above
di = dict(zip(matches, matches_clean))

# Loop through dictionary and call function to combine different versions of subject

for k in di:
    
    subs = [k]
    
    dfu = combine_subs_name(dfu, di[k], subs)

In [39]:
subls = sorted(dfu['subject'].unique())

tag = '**'

# Look for items with tag in the namme
matches = [match for match in subls if tag in match]

# Remove this from the name
matches_clean = [s.replace(tag, '') for s in matches]

# Create dictionary of the two lists above
di = dict(zip(matches, matches_clean))

# Loop through dictionary and call function to combine different versions of subject

for k in di:
    
    subs = [k]
    
    dfu = combine_subs_name(dfu, di[k], subs)

In [40]:
# Combine subject names

subs = ['English and Communication']
dfu = combine_subs_name(dfu, 'English', subs)

subs = ['History (Traditional)', 'History (Alternative)']
dfu = combine_subs_name(dfu, 'History', subs)

subs = ['Secretarial Studies (Audio-typewriting)', 'Secretarial Studies (Word Processing)']
dfu = combine_subs_name(dfu, 'Secretarial Studies', subs)

subs = ['Computing', 'Computing Studies']
dfu = combine_subs_name(dfu, 'Computing Science', subs)

subs = ['Gáidhlig', 'Gàidhlig', 'Gaelic (Native Speakers)']
dfu = combine_subs_name(dfu, 'Gaidhlig', subs)

subs = ['Practical Cookery']
dfu = combine_subs_name(dfu, 'Hospitality: Practical Cookery', subs)

subs = ['Practical Cake Craft']
dfu = combine_subs_name(dfu, 'Hospitality: Practical Cake Craft', subs)

subs = ['Fashion and Textile Technology', 'Home Economics (Fabrics and Fashion)', 
                        'Home Economics (Fabric and Fashion)']
dfu = combine_subs_name(dfu, 'Home Economics: Fashion and Textile Technology', subs)

subs = ['Art and Design (Design)', 'Art and Design Enquiry: Design']
dfu = combine_subs_name(dfu, 'Art and Design: Design', subs)

subs = ['Art and Design (Expressive)', 'Art and Design Enquiry: Expressive']
dfu = combine_subs_name(dfu, 'Art and Design: Expressive', subs)

subs = ['Music (Traditional)', 'Music (Alternative)']
dfu = combine_subs_name(dfu, 'Music', subs)

subs = ['Music: Inventing (Comp)']
dfu = combine_subs_name(dfu, 'Music: Inventing (Composition)', subs)

subs = ['Accounting']
dfu = combine_subs_name(dfu, 'Accounting and Finance', subs)

subs = ['Religious Studies']
dfu = combine_subs_name(dfu, 'Religious, Moral and Philosophical Studies', subs)

subs = ['Administration', 'Office and Information Studies']
dfu = combine_subs_name(dfu, 'Administration and IT', subs)

subs = ['Early Years Care and Education', 'Early Education and Childcare']
dfu = combine_subs_name(dfu, 'Childcare and Development', subs)

subs = ['Anatomy, Physiology and Health']
dfu = combine_subs_name(dfu, 'Human Biology', subs)

subs = ['Home Economics (Food and Nutrition)']
dfu = combine_subs_name(dfu, 'Home Economics: Food and Nutrition', subs)

subs = ['Italian (Traditional)', 'Italian (Alternative)']
dfu = combine_subs_name(dfu, 'Italian', subs)

subs = ['Spanish (Traditional)', 'Spanish (Alternative)']
dfu = combine_subs_name(dfu, 'Spanish', subs)

subs = ['German (Traditional)', 'German (Alternative)']
dfu = combine_subs_name(dfu, 'Spanish', subs)

subs = ['French (Traditional)', 'French (Alternative)']
dfu = combine_subs_name(dfu, 'Spanish', subs)

subs = ['Mandarin (Simplified)', 'Mandarin (Traditional)', 'Cantonese']
dfu = combine_subs_name(dfu, 'Chinese Languages', subs)

subs = ['Hospitality Event Supervision']
dfu = combine_subs_name(dfu, 'Hospitality: Event Supervision', subs)

subs = ['Photography for the Media']
dfu = combine_subs_name(dfu, 'Photography', subs)

subs = ['Dance Practice']
dfu = combine_subs_name(dfu, 'Dance', subs)

subs = ['Media']
dfu = combine_subs_name(dfu, 'Media Studies', subs)

subs = ['Management and Information Studies']
dfu = combine_subs_name(dfu, 'Business Management', subs)

In [41]:
# Remove dodgy values from reading csv process
values = ['Subject', '(continued)']
dfu = dfu.loc[~dfu['subject'].isin(values)]

# Remove writing parts of language courses
values = ['German (Writing)', 'French (Writing)', 'Spanish (Writing)', 'Gaelic (Learners) (Writing)',
             'Italian (Writing)', 'Russian (Writing)', 'Spanish (Writing)', 'Urdu (Writing)']
dfu = dfu.loc[~dfu['subject'].isin(values)]

In [42]:
# Combine subject levels
subs = ['New_Higher', 'Old_Higher']
dfu = combine_subs_level(dfu, 'Higher', subs)

In [44]:
# # Create df for Graph

# # Sort out row order
# dfu = dfu.sort_values(['year', 'level', 'subject', 'grade'])
# dfu.reset_index(drop=True, inplace=True)

# # Look only at entries, passes and fails
# dfgraph = dfu.loc[dfu['grade'].isin(['entries', 'pass', 'na'])].copy()

# # Look only at subjects with a total counts of over 1,000
# dfg = dfgraph.groupby('subject', as_index = False)['count'].sum()
# dfg.drop(dfg.index[dfg['count'] < 1000], inplace = True)
# popular_subs = dfg['subject'].values
# dfgraph = dfgraph.loc[dfu['subject'].isin(popular_subs)].copy()

# # Create a list of subjects and year in correct order
# subjects = sorted(dfgraph['subject'].unique())
# years = sorted(dfgraph['year'].unique())
# grades = ['entries', 'pass', 'na']

# #print(subjects)

In [45]:
# # Plot graph

# levels = ['Advanced_Higher', 'Higher', 'National_5', 'National_4', 'Standard_Grade', 
#                   'Intermediate2', 'Intermediate1']

# fig = px.line(dfgraph, x = "year", y = ["count"], 
#                 color = "subject",
#                 facet_col = 'grade',
#                 facet_row = 'level',
#                 facet_row_spacing = 0.02, 
#                 facet_col_spacing = 0.01, 
#                 width = 1500,
#                 height = 1500,
#                 category_orders = {'subject' : subjects, 
#                                        'year' : years,
#                                           'level' : levels,
#                                               'grade': grades},
#                 labels = {"value": "Count",
#                              "subject": "Subject",
#                                   "level": "Level",
#                                      "grade": "Grade",
#                                          "year": "Year"},
#                 title = 'Total entries, passes, no awards per subject  1986 - 2022'
#                 )

# fig.update_yaxes(rangemode = "tozero")

# fig.update_xaxes(tickangle = 60)

# fig.layout.yaxis.matches = 'y'
# fig.layout.yaxis2.matches = 'y'
# fig.layout.yaxis3.matches = 'y'

# fig.layout.yaxis4.matches = 'y4'
# fig.layout.yaxis5.matches = 'y4'
# fig.layout.yaxis6.matches = 'y4'

# fig.layout.yaxis7.matches = 'y7'
# fig.layout.yaxis8.matches = 'y7'
# fig.layout.yaxis9.matches = 'y7'

# fig.layout.yaxis10.matches = 'y10'
# fig.layout.yaxis11.matches = 'y10'
# fig.layout.yaxis12.matches = 'y10'

# fig.layout.yaxis13.matches = 'y13'
# fig.layout.yaxis14.matches = 'y13'
# fig.layout.yaxis15.matches = 'y13'

# fig.layout.yaxis16.matches = 'y16'
# fig.layout.yaxis17.matches = 'y16'
# fig.layout.yaxis18.matches = 'y16'

# fig.layout.yaxis19.matches = 'y19'
# fig.layout.yaxis20.matches = 'y19'
# fig.layout.yaxis21.matches = 'y19'

# fig.layout.yaxis22.matches = 'y22'
# fig.layout.yaxis23.matches = 'y22'
# fig.layout.yaxis24.matches = 'y22'

# #fig.update_traces(mode = "markers+lines")

# filename = 'total_students_details'

# fig.write_html('./output_graphs/' + filename + '.html')

# fig.show()

In [46]:
# # Setup Chart Studio
# username = 'mrcrookes' # your username
# api_key = 'P54mjVVKJj1xeMfMXaIU' # your api key - go to profile > settings > regenerate key
# chart_studio.tools.set_credentials_file(username = username, api_key = api_key)

# # Upload to Chart Studio
# py.plot(fig, filename = filename, auto_open=True)

In [47]:
dft.to_csv('./output_csvs/1986_2022_data.csv')

In [48]:
dfu.to_csv('./output_csvs/1986_2022_data_simplified.csv')